In [3]:
import cv2 
import numpy as np

#영상 불러오기
obj = cv2.imread('spades.png',cv2.IMREAD_GRAYSCALE) #찾을 이미지
src = cv2.imread('symbols.png',cv2.IMREAD_GRAYSCALE) # 여러 도형


# 객체 영상 외곽선 검출
_, obj_bin  = cv2.threshold(obj,128,255,cv2.THRESH_BINARY_INV)
obj_contours, _ = cv2.findContours(obj_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
obj_pts = obj_contours[0]

# 입력 영상 분석
_, src_bin = cv2.threshold(src,128,255,cv2.THRESH_BINARY_INV)
contours, _ = cv2.findContours(src_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

#결과 영상 
dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

# 입력 영상의 모든 객체 영역에 대해서 
for pts in contours:
    if cv2.contourArea(pts) < 1000:
        continue
    
    rc = cv2.boundingRect(pts)
    cv2.rectangle(dst, rc, (255,0,0),1)
    
    # 모양비교
    dist = cv2.matchShapes(obj_pts, pts, cv2.CONTOURS_MATCH_I3, 0)
    
    cv2.putText(dst, str(round(dist,4)), (rc[0],rc[1]-3 ), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0), 1, cv2.LINE_AA)
    
    if dist < 0.1:
        cv2.rectangle(dst, rc, (0,0,255),2)
        
cv2.imshow('obj',obj)
cv2.imshow('dst',dst)
cv2.waitKey()
cv2.destroyAllWindows()
    


### 템플릿 매칭\
템플릿 매칭(template matching)이란?\
입력 영상에서 (작은 크기의) 템플릿 영상과 일치하는 부분을 찾는 기법\
템플릿: 찾을 대상이 되는 작은 영상. 패치 (patch)\

템플릿 매칭함수\
cv2.matchTemplate(image,templ,method,result=None,mask=None) => res\

image: 입력 영상. \
templ: 템플릿 영상. image보다 같거나 작은 크기, 같은 타입\
method: 비교 방법. cv2.TM_으로 시작하는 플래그 지정\

TM_SQDIFF/ TM_SQDIFF_NORMED=> sum of squared difference\
TM_CCORR/ TM_CCORR_NORMED => Cross Correlation\
TM_CCOEFF/ TM_CCOEFF_NORMED => Correlation Coefficient\
res: 비교 결과 행렬. numpy.ndarray.type = numpy.float32, \
image의 크기가 w*h이고 templ의 크기가 w*h이면 \
result의 크기 (W-w+1)*(H-h+1)

함수 반환값 res처리\
cv2.minMaxLoc()는 배열에서 가장 큰값, 작은 값과 그 위치 반환\
min_val, max_val, min_loc ,max_loc = cv2.minMaxLoc(res)